**Langkah 1: Menyusun Data Pelamar**

Data pelamar disusun dalam bentuk list of dictionaries di mana setiap dictionary mewakili satu pelamar dan atributnya:

IPK: Tingkat kelulusan akademis (Bagus, Cukup, Kurang).

Psikologi: Hasil tes psikologi (Tinggi, Sedang, Rendah).

Wawancara: Penilaian hasil wawancara (Baik, Buruk).
Diterima: Apakah pelamar diterima atau tidak berdasarkan atribut di atas (Ya, Tidak).

In [14]:
import math

# Data pelamar
data = [
    {'Pelamar': 'A', 'IPK': 'Bagus', 'Psikologi': 'Tinggi', 'Wawancara': 'Buruk', 'Diterima': 'Tidak'},
    {'Pelamar': 'B', 'IPK': 'Bagus', 'Psikologi': 'Rendah', 'Wawancara': 'Baik', 'Diterima': 'Ya'},
    {'Pelamar': 'C', 'IPK': 'Cukup', 'Psikologi': 'Tinggi', 'Wawancara': 'Buruk', 'Diterima': 'Tidak'},
    {'Pelamar': 'D', 'IPK': 'Cukup', 'Psikologi': 'Rendah', 'Wawancara': 'Baik', 'Diterima': 'Ya'},
    {'Pelamar': 'E', 'IPK': 'Kurang', 'Psikologi': 'Tinggi', 'Wawancara': 'Buruk', 'Diterima': 'Tidak'},
    {'Pelamar': 'F', 'IPK': 'Kurang', 'Psikologi': 'Sedang', 'Wawancara': 'Baik', 'Diterima': 'Ya'},
    {'Pelamar': 'G', 'IPK': 'Kurang', 'Psikologi': 'Rendah', 'Wawancara': 'Buruk', 'Diterima': 'Tidak'},
]


**Langkah 2: Menghitung Entropi**

Entropi adalah ukuran ketidakpastian atau ketidakteraturan dalam suatu dataset. Pada algoritma ID3, kita menghitung entropi untuk atribut target (dalam hal ini, Diterima) untuk mengukur seberapa murni atau tidak murninya data tersebut. Semakin tinggi entropi, semakin besar ketidakpastian data.

Fungsi entropy menghitung entropi dari dataset berdasarkan distribusi dari atribut target (Diterima).

In [23]:
def entropy(data, target_attr):
    values = [record[target_attr] for record in data]  # Ambil semua nilai target ('Diterima')
    value_freq = {}
    data_entropy = 0.0

    # Hitung frekuensi setiap nilai dalam target attribute
    for value in values:
        if value in value_freq:
            value_freq[value] += 1.0
        else:
            value_freq[value] = 1.0

    # Hitung entropi
    for freq in value_freq.values():
        prob = freq / len(data)  # Probabilitas setiap nilai
        data_entropy += -prob * math.log2(prob)  # Entropi total

    return data_entropy


**Langkah 3: Menghitung Information Gain**

Information Gain (IG) mengukur pengurangan entropi setelah membagi dataset berdasarkan suatu atribut. Semakin besar IG, semakin baik atribut tersebut untuk digunakan sebagai pembagi.

In [16]:
def information_gain(data, attr, target_attr):
    attr_values = [record[attr] for record in data]  # Nilai-nilai atribut yang dipilih
    value_freq = {}
    subset_entropy = 0.0

    # Hitung frekuensi setiap nilai dari atribut yang dipilih
    for value in attr_values:
        if value in value_freq:
            value_freq[value] += 1.0
        else:
            value_freq[value] = 1.0

    # Hitung entropi untuk setiap subset data
    for value, freq in value_freq.items():
        prob = freq / len(data)
        subset = [record for record in data if record[attr] == value]  # Subset dari data yang dibagi oleh nilai ini
        subset_entropy += prob * entropy(subset, target_attr)

    # Hitung information gain
    total_entropy = entropy(data, target_attr)
    return total_entropy - subset_entropy


Langkah 4: Membuat Decision Tree

Untuk membangun decision tree, kita melakukan proses rekursif:

1. Pilih atribut dengan information gain tertinggi sebagai akar dari decision tree.
2. Bagi dataset menjadi subset berdasarkan nilai dari atribut tersebut.
3. Ulangi langkah ini untuk setiap subset hingga tidak ada lagi atribut yang bisa dipilih atau semua data dalam subset memiliki nilai target yang sama.

In [17]:
def choose_best_attribute(data, attributes, target_attr):
    """Memilih atribut dengan information gain tertinggi."""
    best_gain = 0.0
    best_attr = None

    for attr in attributes:
        gain = information_gain(data, attr, target_attr)
        if gain > best_gain:
            best_gain = gain
            best_attr = attr

    return best_attr

def create_decision_tree(data, attributes, target_attr):
    target_values = [record[target_attr] for record in data]

    # Jika semua nilai target sama, kembalikan nilai itu (base case)
    if target_values.count(target_values[0]) == len(target_values):
        return target_values[0]

    # Jika tidak ada atribut yang tersisa, kembalikan nilai mayoritas
    if not attributes:
        return majority_value(data, target_attr)

    # Pilih atribut terbaik untuk memecah data
    best_attr = choose_best_attribute(data, attributes, target_attr)

    # Buat pohon (dictionary)
    tree = {best_attr: {}}

    # Hapus atribut yang sudah dipilih
    remaining_attrs = [attr for attr in attributes if attr != best_attr]

    # Buat cabang untuk setiap nilai dari atribut terbaik
    for value in set(record[best_attr] for record in data):
        subset = [record for record in data if record[best_attr] == value]
        subtree = create_decision_tree(subset, remaining_attrs, target_attr)
        tree[best_attr][value] = subtree

    return tree


**Langkah 5: Menjalankan dan Menampilkan Decision Tree**

Membangun dan menampilkan decision tree berdasarkan data penerimaan karyawan. Atribut yang tersedia adalah IPK, Psikologi, dan Wawancara.

In [24]:
# Daftar atribut yang tersedia (kecuali atribut target 'Diterima')
attributes = ['IPK', 'Psikologi', 'Wawancara']

# Buat decision tree
decision_tree = create_decision_tree(data, attributes, 'Diterima')

# Cetak decision tree
import pprint
pprint.pprint(decision_tree)


{'Wawancara': {'Baik': 'Ya', 'Buruk': 'Tidak'}}
